In [56]:
import pandas as pd
import numpy as np
import re

# Helper function

In [57]:
def data_cleaning(data,col_1):
    data[col_1]=data[col_1].apply(lambda x:x.lower())
    # Set of all special characters
    exclude = set(string.punctuation) 
    data[col_1] = data[col_1].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
    # Remove extra spaces
    data[col_1] = data[col_1].apply(lambda x: x.strip())
    data[col_1] = data[col_1].apply(lambda x: re.sub(" +", " ", x))
    return data[col_1]
# Add start and end tokens to target sequences
# data['sql'] = data['sql'].apply(lambda x : 'START_ '+ x + ' _END')

In [58]:
df = pd.read_csv('../input/train-data/train.csv')

In [59]:
df =  df.sample(n=40000,random_state=1)

In [60]:
df.head()

In [61]:
df['question'] = data_cleaning(df,'question')

In [62]:
df['sql'] = df['sql'].apply(lambda x : 'START_ '+ x + ' _END')

In [63]:
data = pd.DataFrame()
data['input_data'] = df['question']
data['sql'] = df['sql']
df = data
df.head()

In [64]:
df['input_sent_length'] = df['input_data'].apply(lambda x:len(x.split()))
df['sql_sent_length'] = df['sql'].apply(lambda x:len(x.split()))
df.head()

In [65]:
df.describe()

In [66]:
max(df['input_sent_length'])

In [67]:
max_input_sen_length = max(df['input_sent_length'])
max_output_sen_length = max(df['sql_sent_length'])

# rough-1


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
all_eng_words=set()
for eng in df['sql']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
t=Tokenizer(num_words=4000)
t.fit_on_texts(df['input_data'])
print(t.word_counts) # how many times it appear
print(t.word_index) # indexing words from vocab
print(t.word_docs)  # how many times appear in each doc
input_seq = t.texts_to_sequences(np.array(df['input_data']))
input_pad_seq = pad_sequences(input_seq,maxlen=max_input_sen_length,padding='post')
print(t.sequences_to_texts([input_pad_seq[0]]))
print(input_pad_seq[0])
sql_token = Tokenizer(4000)
sql_token.fit_on_texts(df['sql'])
print(sql_token.word_counts) # how many times it appear
print(sql_token.word_index) # indexing words from vocab
print(sql_token.word_docs)  # how many times appear in each doc
output_seq = sql_token.texts_to_sequences(np.array(df['sql']))
output_pad_seq = pad_sequences(output_seq,maxlen=max_input_sen_length,padding='post')
print(sql_token.sequences_to_texts([output_pad_seq[0]]))
print(output_pad_seq[0])



# rough-2


import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras import regularizers
from keras import layers
from keras.layers import Embedding, Bidirectional, GlobalMaxPool1D
from gensim.models import KeyedVectors
from keras import regularizers
from keras import layers
from keras.layers import Embedding,RepeatVector
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense,Flatten,Conv2D,Conv1D,GlobalMaxPooling1D
from keras.layers import Input, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from keras.callbacks import EarlyStopping
encoder_inp = Input(shape=(max_input_sen_length,))
encoder_embed = Embedding(4000,100,input_length=max_input_sen_length)(encoder_inp)
encoder_lstm_cell = LSTM(64,return_state=True)
encoder_output,encoder_state_h,encoder_state_c = encoder_lstm_cell(encoder_embed)
Creating LSTM decoder model and feeding the output states (h,c) of lstm of encoders
decoder_inp   = Input(shape=(max_input_sen_length,))
decoder_embed = Embedding(4000,100,input_length=max_input_sen_length)(decoder_inp)
decoder_lstm_cell = LSTM(64,return_sequences='True',return_state=True)
decoder_output,decoder_state_h,decoder_state_c = decoder_lstm_cell(decoder_embed,initial_state=[encoder_state_h,encoder_state_c])
decoder_dense_cell1 = Dense(max_input_sen_length,activation='relu')
decoder_d_output    = decoder_dense_cell1(decoder_output)
decoder_dense_cell2 = Dense(max_input_sen_length,activation='softmax')
decoder_output = decoder_dense_cell2(decoder_d_output)
model = Model([encoder_inp,decoder_inp],decoder_output) 
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

# Model

In [68]:
from keras.preprocessing.text import one_hot
import string
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding,Bidirectional
from keras.losses import sparse_categorical_crossentropy

In [69]:
def token(sentances):
  # create tokenization
  text_token =Tokenizer()
  text_token.fit_on_texts(sentances)
  return text_token.texts_to_sequences(sentances),text_token

In [70]:
df_arr = np.array(df)

In [71]:
input_text_tokenized,input_text_tokenizer = token(df_arr[:,0])
output_text_tokenized,output_text_tokenizer = token(df_arr[:,1])

In [72]:
print('Maximum length spanish sentence: {}'.format(len(max(input_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(output_text_tokenized,key=len))))

In [73]:
# Check language length
input_vocab = len(input_text_tokenizer.word_index) + 1
output_vocab = len(output_text_tokenizer.word_index) + 1
print("input_vocab vocabulary is of {} unique words".format(input_vocab))
print("output_vocab vocabulary is of {} unique words".format(output_vocab))

In [74]:
max_input_len = int(len(max(input_text_tokenized,key=len)))
max_output_len = int(len(max(output_text_tokenized,key=len)))

input_pad_sentence = pad_sequences(input_text_tokenized, max_input_len, padding = "post")
output_pad_sentence = pad_sequences(output_text_tokenized, max_output_len, padding = "post")

# Reshape data
input_pad_sentence = input_pad_sentence.reshape(*input_pad_sentence.shape, 1)
output_pad_sentence = output_pad_sentence.reshape(*output_pad_sentence.shape, 1)

In [75]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(input_pad_sentence,output_pad_sentence,test_size=0.05,random_state=2)

In [76]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [77]:
from keras.backend import clear_session
clear_session()
input_sequence = Input(shape=(max_input_len,))
embedding = Embedding(input_dim=input_vocab, output_dim=128,)(input_sequence)
encoder = Bidirectional(LSTM(64, return_sequences=False))(embedding)
r_vec = RepeatVector(max_output_len)(encoder)
decoder = Bidirectional(LSTM(64, return_sequences=True, dropout=0.2))(r_vec)
logits = TimeDistributed(Dense(output_vocab))(decoder)
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()
ep = 100
batch_size=128
model_results = enc_dec_model.fit(x_train, y_train, batch_size=batch_size, epochs=ep,validation_data=(x_test,y_test),callbacks=[callback])

In [78]:
input_sentences = df_arr[:,0]
output_sentences = df_arr[:,1]
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 
    sen = ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
    sen = sen.replace('<empty>','')
    sen = sen.replace('start','')
    sen = sen.replace('end','')
    return sen



In [79]:
for i in [81,47,75,4781,4775]:
    print("The Input sentence is: {}".format(input_sentences[i]))
    print("The SQL Query is: {}".format(output_sentences[i]))
    print('The predicted SQL Query is :')
    print(logits_to_sentence(enc_dec_model.predict(input_pad_sentence[i:i+1])[0], output_text_tokenizer))
    print('--'*20)

In [80]:
enc_dec_model.save('best_wikisql_model.h5')

In [81]:
import seaborn as sns


In [83]:
sns.lineplot(x=range(0,40),y=model_results.history['val_loss'],label='val')
sns.lineplot(x=range(0,40),y=model_results.history['loss'],label='Train')

In [84]:
sns.lineplot(x=range(0,40),y=model_results.history['val_accuracy'],label='val')
sns.lineplot(x=range(0,40),y=model_results.history['accuracy'],label='Train')